In [3]:
import spacy
nlp = spacy.load("es_core_news_md")

In [20]:
# Diccionario de palabras con su polaridad (-1 a +1)
polaridad = {
    "bueno": 1.0,
    "excelente": 1.0,
    "malo": -1.0,
    "horrible": -1.0,
    "gusta": 0.8,
    "encanta": 1.0,
    "odio": -1.0,
    "terrible": -1.0,
    "desastre": -1.0,
    "no": -0.5,
    "nunca": -0.5,
    "siempre": 0.5,
    "fantástico": 1.0,
    "horrendo": -1.0
}

# Intensificadores y atenuadores
intensificadores = {
    "muy": 1.5,
    "bastante": 1.3,
    "sumamente": 1.7
}

atenuadores = {
    "algo": 0.7,
    "un_poco": 0.5,
}


In [21]:
def sentimiento_token(token):
    """
    Calcula la polaridad de un token según su contexto gramatical.
    """
    base = polaridad.get(token.lemma_, 0)
    if base == 0:
        return 0

    # Ajuste por negación
    for hijo in token.children:
        if hijo.dep_ == "neg":
            base *= -1

    # Ajuste por modificadores adverbiales (intensificadores o atenuadores)
    for hijo in token.children:
        if hijo.dep_ == "advmod":
            if hijo.lemma_ in intensificadores:
                base *= intensificadores[hijo.lemma_]
            elif hijo.lemma_ in atenuadores:
                base *= atenuadores[hijo.lemma_]

    return base


In [4]:
def analizar_sentimiento(texto):
    doc = nlp(texto)
    polaridades = []
    
    for sent in doc.sents:  # Recorremos cada oración
        total = 0
        count = 0
        
        for token in sent:
            p = sentimiento_token(token)
            if p != 0:
                total += p
                count += 1
        
        # Promedio de sentimiento en la oración
        if count > 0:
            polaridad_media = total / count
        else:
            polaridad_media = 0

        # Si la oración tiene "pero", ponderamos la parte posterior
        if any(tok.text.lower() == "pero" for tok in sent):
            polaridad_media *= 1.2  # damos más peso a lo posterior

        polaridades.append(polaridad_media)
    
    # Composición global del texto
    if len(polaridades) > 0:
        global_sentimiento = sum(polaridades) / len(polaridades)
    else:
        global_sentimiento = 0
    
    # Clasificación final
    if global_sentimiento > 0.25:
        etiqueta = "Positivo"
    elif global_sentimiento < -0.25:
        etiqueta = "Negativo"
    else:
        etiqueta = "Neutro"

    return {
        "sentimiento_global": round(global_sentimiento, 2),
        "etiqueta": etiqueta,
        "detalles": polaridades
    }


In [8]:
texto = "No me gusta el servicio, pero la comida está muy buena."
resultado = analizar_sentimiento(texto)
print(resultado)


{'sentimiento_global': -0.6, 'etiqueta': 'Negativo', 'detalles': [-0.6]}


In [36]:
# Renderizar con displacy sin depender de IPython.display.display (que falta)
from spacy import displacy
import spacy
nlp = spacy.load('es_core_news_md')
doc = nlp("No me gusta el servicio, pero la comida está muy buena.")
# Generar HTML con displacy
html = displacy.render(doc, style='dep', jupyter=False)

# Intentar usar display_html (presente en IPython.core.display en esta venv)
try:
    from IPython.core import display as ipd
    ipd.display_html(html, raw=True)
except Exception as e:
    # Fallback: escribir a archivo para abrir en el navegador
    with open('displacy_output.html', 'w', encoding='utf8') as f:
        f.write(html)
        print('No se pudo mostrar en el notebook:', e)
        print("Se ha escrito 'displacy_output.html' en el directorio de trabajo.")

No 
 ADV 
 

 
 me 
 PRON 
 

 
 gusta 
 VERB 
 

 
 el 
 DET 
 

 
 servicio, 
 NOUN 
 

 
 pero 
 CCONJ 
 

 
 la 
 DET 
 

 
 comida 
 NOUN 
 

 
 está 
 AUX 
 

 
 muy 
 ADV 
 

 
 buena. 
 ADJ 
 

 
 
 
 advmod 
 
 
 

 
 
 
 iobj 
 
 
 

 
 
 
 det 
 
 
 

 
 
 
 nsubj 
 
 
 

 
 
 
 cc 
 
 
 

 
 
 
 det 
 
 
 

 
 
 
 nsubj 
 
 
 

 
 
 
 cop 
 
 
 

 
 
 
 advmod 
 
 
 

 
 
 
 conj

In [27]:
import spacy
nlp = spacy.load("es_core_news_md")

# Ejemplo de diccionario (asegúrate de usar lemas en minúsculas)
polaridad = {
    "bueno": 1.0,
    "excelente": 1.0,
    "malo": -1.0,
    "horrible": -1.0,
    "gustar": 0.8,
    "encantar": 1.0,
    "odiar": -1.0,
    "terrible": -1.0,
    "desastre": -1.0,
    # "no": -0.5,
    # "nunca": -0.5,
    # "siempre": 0.5,
    # "fantástico": 1.0,
    # "horrendo": -1.0
}

intensificadores = {"muy": 1.5, "bastante": 1.3, "sumamente": 1.7}
atenuadores = {"algo": 0.7, "un_poco": 0.5, "poco": 0.6}

def _lookup_lemma_polarity(token):
    """Buscar polaridad exacta por lema (minúscula)."""
    return polaridad.get(token.lemma_.lower(), 0.0)

def _search_subtree_polarity(token, max_depth=2, visited=None):
    """
    Busca polaridad en token, sus hijos y (si aplica) nietos hasta max_depth.
    Devuelve (polaridad, fuente_token) o (0, None).
    """
    if visited is None:
        visited = set()
    if (token.i, token.sent.start) in visited:
        return 0.0, None
    visited.add((token.i, token.sent.start))

    p = _lookup_lemma_polarity(token)
    if p != 0.0:
        return p, token

    if max_depth <= 0:
        return 0.0, None

    # Priorizar adjetivos que modifican sustantivos (amod), complementos, objetos
    for child in token.children:
        # Buscar adjetivos que califican (amod), o complementos (xcomp, ccomp, obj)
        if child.dep_ in {"amod", "advmod", "xcomp", "ccomp", "obj", "iobj", "acomp"}:
            p_child, src = _search_subtree_polarity(child, max_depth-1, visited)
            if p_child != 0.0:
                return p_child, src

    # Si no halló en hijos prioritarios, buscar entre todos los hijos
    for child in token.children:
        p_child, src = _search_subtree_polarity(child, max_depth-1, visited)
        if p_child != 0.0:
            return p_child, src

    return 0.0, None

def _has_negation_affecting(token):
    """
    Detecta negación que afecte al token: puede estar en sus hijos (dep_='neg')
    o en sus ancestros directos (p.ej. 'no' que modifica el verbo superior).
    """
    # hijos con dep_ == neg
    for ch in token.children:
        if ch.dep_ == "neg" or ch.lemma_.lower() in {"no", "nunca", "jamás"}:
            return True
    # ancestros con negación (por ejemplo, la negación puede estar sobre el verbo principal)
    for anc in token.ancestors:
        for ch in anc.children:
            if ch.dep_ == "neg" or ch.lemma_.lower() in {"no", "nunca", "jamás"}:
                # si la negación está en un ancestro cercano, la consideramos
                return True
    return False

def sentimiento_token(token):
    """
    Calcula la polaridad asociada a este token de forma contextual:
    - intenta obtener polaridad del propio token (por lema)
    - si no la tiene, busca en su subtree (adjetivos que califican, complementos)
    - aplica negación y modificadores (advmod)
    - en caso de tokens que funcionan como determinantes/modificadores, consulta el head
    """
    # 1) intentar polaridad directa
    base = _lookup_lemma_polarity(token)

    source_token = None
    if base != 0.0:
        source_token = token
    else:
        # 2) buscar en subtree (hijos que puedan portar polaridad)
        base, source_token = _search_subtree_polarity(token, max_depth=2)

    # 3) si todavía 0, intentar tomar la polaridad del head (ej: DET -> NOUN, o token dentro de frase)
    if base == 0.0 and token.head is not None and token.head != token:
        base = _lookup_lemma_polarity(token.head)
        if base == 0.0:
            # también buscar en subtree del head
            base, source_token = _search_subtree_polarity(token.head, max_depth=1)

    # Si no hay polaridad encontrada, devolvemos 0 (pero sin salir prematuramente)
    if base == 0.0:
        return 0.0

    # 4) detectar y aplicar negación que afecte al token/palabra fuente
    # chequeamos negación sobre el token original y sobre la fuente encontrada
    neg = _has_negation_affecting(token)
    if source_token is not None and not neg:
        neg = _has_negation_affecting(source_token)
    if neg:
        base *= -1

    # 5) aplicar modificadores (intensificadores/atenuadores) que sean hijos del source_token o del token
    modifiers_sources = [token]
    if source_token is not None and source_token != token:
        modifiers_sources.append(source_token)

    for src in modifiers_sources:
        for ch in src.children:
            if ch.dep_ == "advmod" or ch.pos_ == "ADV":
                lemma = ch.lemma_.lower()
                if lemma in intensificadores:
                    base *= intensificadores[lemma]
                elif lemma in atenuadores:
                    base *= atenuadores[lemma]

    return base


In [26]:
def dividir_por_conjunciones(texto):
    """
    Divide el texto en suboraciones según conjunciones adversativas (pero, aunque, sin embargo...).
    """
    doc = nlp(texto)
    suboraciones = []
    start = 0

    for token in doc:
        if token.text.lower() in {"pero", "aunque", "sin embargo", "sin embargo,"}:
            sub = doc[start:token.i].text.strip(", ")
            if sub:
                suboraciones.append(sub)
            start = token.i + 1

    # agregar la última parte
    final = doc[start:].text.strip(", ")
    if final:
        suboraciones.append(final)

    return suboraciones


In [31]:
def analizar_sentimiento(texto):
    suboraciones = dividir_por_conjunciones(texto)
    if not suboraciones:
        suboraciones = [texto]

    polaridades = []
    for sub in suboraciones:
        doc = nlp(sub)
        total = 0
        count = 0
        for token in doc:
            p = sentimiento_token(token)
            if p != 0:
                total += p
                count += 1
        polaridad_media = total / count if count > 0 else 0
        polaridades.append(polaridad_media)
    
    # Composición global del texto
    if len(polaridades) > 0:
        global_sentimiento = sum(polaridades) / len(polaridades)
    else:
        global_sentimiento = 0
    
    # Clasificación final
    if global_sentimiento > 0.25:
        etiqueta = "Positivo"
    elif global_sentimiento < -0.25:
        etiqueta = "Negativo"
    else:
        etiqueta = "Neutro"

    return {
        "sentimiento_global": round(global_sentimiento, 2),
        "etiqueta": etiqueta,
        "detalles": polaridades
    }


In [32]:
texto = "No me gusta el servicio, pero la comida está muy buena."
resultado = analizar_sentimiento(texto)
print(resultado)


{'sentimiento_global': -0.4, 'etiqueta': 'Negativo', 'detalles': [-0.8, 0]}


In [3]:
# --- Diccionarios léxicos ---
polaridad = {
    "bueno": 1.0,
    "excelente": 1.5,
    "positivo": 1.0,
    "agradable": 1.0,
    "malo": -1.0,
    "terrible": -1.5,
    "horrible": -1.5,
    "deficiente": -0.8,
    "gusta": 0.8,
    "encanta": 1.2,
    "odia": -1.2,
    "mal": -0.8,
    "feo": -1.0,
    "bonito": 0.8,
    "delicioso": 1.3,
    "asqueroso": -1.3
}

intensificadores = {
    "muy": 1.5,
    "bastante": 1.3,
    "poco": 0.5,
    "algo": 0.8
}

# --- 1. Dividir por conjunciones adversativas ---
def dividir_por_conjunciones(texto):
    doc = nlp(texto)
    suboraciones = []
    conjunciones = []
    start = 0

    for token in doc:
        if token.text.lower() in {"pero", "aunque", "sin embargo"}:
            sub = doc[start:token.i].text.strip(", ")
            if sub:
                suboraciones.append(sub)
                conjunciones.append(token.text.lower())
            start = token.i + 1

    final = doc[start:].text.strip(", ")
    if final:
        suboraciones.append(final)

    return suboraciones, conjunciones


# --- 2. Analizar el sentimiento de un token ---
def sentimiento_token(token):
    # Polaridad base si existe
    base = polaridad.get(token.lemma_, None)

    # Si no hay polaridad léxica, no devolvemos nada todavía
    if base is None:
        return 0

    # Negaciones
    for hijo in token.children:
        if hijo.dep_ == "neg" or hijo.lemma_ == "no":
            base *= -1

    # Intensificadores
    for hijo in token.children:
        if hijo.pos_ == "ADV" and hijo.lemma_ in intensificadores:
            base *= intensificadores[hijo.lemma_]

    # También si este token es modificado por un adverbio (e.g., “muy buena”)
    for tok in token.head.children:
        if tok.pos_ == "ADV" and tok.lemma_ in intensificadores and tok.head == token:
            base *= intensificadores[tok.lemma_]

    return base


# --- 3. Combinar polaridades según conjunciones ---
def combinar_polaridades(polaridades, conjunciones):
    if not polaridades:
        return 0

    # Si hay "pero" o "aunque", priorizar la última parte
    if any(c in {"pero", "sin embargo", "aunque"} for c in conjunciones):
        return polaridades[-1]
    else:
        # promedio simple si no hay contraste
        return sum(polaridades) / len(polaridades)


# --- 4. Analizar sentimiento global ---
def analizar_sentimiento(texto):
    suboraciones, conjunciones = dividir_por_conjunciones(texto)
    if not suboraciones:
        suboraciones = [texto]

    polaridades = []
    for sub in suboraciones:
        doc = nlp(sub)
        total = 0
        count = 0
        for token in doc:
            p = sentimiento_token(token)
            if p != 0:
                total += p
                count += 1
        polaridad_media = total / count if count > 0 else 0
        polaridades.append(polaridad_media)

    polaridad_final = combinar_polaridades(polaridades, conjunciones)

    etiqueta = (
        "Positivo" if polaridad_final > 0.1 else
        "Negativo" if polaridad_final < -0.1 else
        "Neutro"
    )

    return {
        "sentimiento_global": round(polaridad_final, 2),
        "etiqueta": etiqueta,
        "detalles": [round(p, 2) for p in polaridades]
    }


# --- Ejemplo ---
texto = "No me gusta el servicio, pero la comida está muy buena."
resultado = analizar_sentimiento(texto)
print(resultado)


{'sentimiento_global': 0, 'etiqueta': 'Neutro', 'detalles': [0, 0]}


In [4]:
# --- Léxicos (lemmas en minúscula) ---
polaridad = {
    "bueno": 1.0, "excelente": 1.5, "agradable": 1.0,
    "malo": -1.0, "terrible": -1.5, "horrible": -1.5,
    "deficiente": -0.8, "gustar": 0.8, "encantar": 1.2,
    "odiar": -1.2, "mal": -0.8, "feo": -1.0,
    "bonito": 0.8, "delicioso": 1.3, "asqueroso": -1.3
}

intensificadores = {"muy": 1.5, "bastante": 1.3, "sumamente": 1.7}
atenuadores = {"poco": 0.6, "algo": 0.8, "un_poco": 0.7}

# Conectores adversativos y su peso para priorizar la cláusula posterior
ADVERSATIVE_WEIGHTS = {
    "pero": 0.7,
    "sin embargo": 0.75,
    "aunque": 0.6,
    "no obstante": 0.7
}

# ---------- utilidades de búsqueda en árbol ----------
def _lookup_polarity_by_lemma(token):
    return polaridad.get(token.lemma_.lower(), 0.0)

def _search_subtree_polarity(token, max_depth=2, visited=None):
    """
    Busca polaridad en token y en hijos relevantes hasta max_depth.
    Devuelve (p, source_token) donde p puede ser 0.0 si no halló.
    """
    if visited is None:
        visited = set()
    key = (token.i, token.sent.start)
    if key in visited:
        return 0.0, None
    visited.add(key)

    p = _lookup_polarity_by_lemma(token)
    if abs(p) > 1e-9:
        return p, token

    if max_depth <= 0:
        return 0.0, None

    # hijos prioritarios
    for child in token.children:
        if child.dep_ in {"amod", "advmod", "acomp", "xcomp", "ccomp", "obj", "iobj"}:
            p_child, src = _search_subtree_polarity(child, max_depth - 1, visited)
            if abs(p_child) > 1e-9:
                return p_child, src

    # luego buscar más ampliamente entre hijos
    for child in token.children:
        p_child, src = _search_subtree_polarity(child, max_depth - 1, visited)
        if abs(p_child) > 1e-9:
            return p_child, src

    return 0.0, None

def _has_negation_affecting(token):
    # busca 'neg' en hijos directos
    for ch in token.children:
        if ch.dep_ == "neg" or ch.lemma_.lower() in {"no", "nunca", "jamás"}:
            return True
    # busca negación en ancestros cercanos
    for anc in token.ancestors:
        for ch in anc.children:
            if ch.dep_ == "neg" or ch.lemma_.lower() in {"no", "nunca", "jamás"}:
                # si la negación está en ancestro cercano, la consideramos
                # limitamos a ancestros cercanos: depth implícito por la naturaleza del árbol
                return True
    return False

def _apply_modifiers(base, source_token, token):
    """
    Aplica intensificadores/atenuadores encontrados como hijos del source_token o del token.
    """
    if base == 0:
        return 0.0

    # revisar hijos del source
    for ch in list(source_token.children) + list(token.children):
        if ch.pos_ == "ADV":
            lemma = ch.lemma_.lower()
            if lemma in intensificadores:
                base *= intensificadores[lemma]
            elif lemma in atenuadores:
                base *= atenuadores[lemma]
    # revisar adverbios que estén en el head (caso "muy buena" a veces estructura diferente)
    for ch in source_token.head.children:
        if ch.pos_ == "ADV" and ch.head == source_token:
            lemma = ch.lemma_.lower()
            if lemma in intensificadores:
                base *= intensificadores[lemma]
            elif lemma in atenuadores:
                base *= atenuadores[lemma]

    return base

# ---------- análisis de una región/span ----------
def sentimiento_para_span(span):
    """
    Analiza un span (suboración) usando dependencia. 
    Busca fuentes de polaridad en el árbol y compone un valor medio.
    """
    encontrados = {}  # source_token.i -> polaridad (evita duplicados)
    for token in span:
        # intentar encontrar polaridad en token o en su subtree/head
        p, src = _search_subtree_polarity(token, max_depth=2)
        if abs(p) < 1e-9:
            # intentar en el head si no se encontró
            if token.head is not None and token.head != token:
                p, src = _search_subtree_polarity(token.head, max_depth=1)
        if abs(p) < 1e-9:
            continue

        # aplicar negación relativa al token y al source
        neg = _has_negation_affecting(token)
        if src is not None and not neg:
            neg = _has_negation_affecting(src)
        if neg:
            p *= -1

        # aplicar modificadores
        if src is None:
            src = token
        p = _apply_modifiers(p, src, token)

        # almacenar (evitar contar mismo source dos veces)
        if src.i not in encontrados:
            encontrados[src.i] = p

    if not encontrados:
        return 0.0, []
    vals = list(encontrados.values())
    # retorno promedio simple de fuentes encontradas en este span
    return sum(vals) / len(vals), vals

# ---------- división por conjunciones (mejorada) ----------
def dividir_por_conjunciones_doc(doc):
    """Divide el Doc en sub-spans por conjunciones adversativas detectadas."""
    adversatives = set(ADVERSATIVE_WEIGHTS.keys())
    # buscar tokens que actúen como conectores
    boundaries = []
    for token in doc:
        text_low = token.text.lower()
        # tratar multi-word connectors (sin embargo) mirando token+next
        two = (token.text + " " + (token.nbor(1).text if token.i+1 < len(doc) else "")).lower()
        if text_low in adversatives:
            boundaries.append((token.i, token.text.lower()))
        elif two in adversatives:
            boundaries.append((token.i, two))
    # si no hay boundaries, return whole doc as one span
    if not boundaries:
        return [doc], []
    spans = []
    conj_vals = []
    start = 0
    for (idx, conj_text) in boundaries:
        # span is doc[start: idx]
        span = doc[start:idx]
        if span.text.strip():
            spans.append(span)
            conj_vals.append(conj_text)
        # move start to after connector token (one or two tokens)
        if " " in conj_text:
            start = idx + 2
        else:
            start = idx + 1
    # last span
    lastspan = doc[start: len(doc)]
    if lastspan.text.strip():
        spans.append(lastspan)
    return spans, conj_vals

# ---------- combinar polaridades con ponderación ----------
def combinar_polaridades(polaridades, conjunciones):
    if not polaridades:
        return 0.0
    if not conjunciones:
        return sum(polaridades) / len(polaridades)
    # si hay varios, aplicamos la regla local: cada conjuncion pondera la parte posterior
    # asumimos que polaridades list está en orden y conjunciones apunta a las separaciones anteriores
    # simplificamos: si hay al menos una conj adversativa, aplicamos peso a la última cláusula según el conector detectado
    last_conj = conjunciones[-1]
    weight_last = ADVERSATIVE_WEIGHTS.get(last_conj, 0.7)
    # weight_last aplica a la última polaridad
    if len(polaridades) == 1:
        return polaridades[0]
    # combinar: last * w + rest_mean * (1-w)
    rest_mean = sum(polaridades[:-1]) / len(polaridades[:-1])
    return polaridades[-1] * weight_last + rest_mean * (1 - weight_last)

# ---------- función principal ----------
def analizar_sentimiento(texto):
    doc = nlp(texto)
    spans, conjunciones = dividir_por_conjunciones_doc(doc)
    polaridades = []
    detalles = []
    for span in spans:
        p_span, fuentes = sentimiento_para_span(span)
        polaridades.append(round(p_span, 4))
        detalles.append([round(x,4) for x in fuentes])
    p_final = combinar_polaridades(polaridades, conjunciones)
    etiqueta = "Positivo" if p_final > 0.1 else ("Negativo" if p_final < -0.1 else "Neutro")
    return {
        "sentimiento_global": round(p_final, 4),
        "etiqueta": etiqueta,
        "detalles_por_span": detalles,
        "polaridades_spans": polaridades,
        "conjunciones": conjunciones
    }

# ---------- ejemplo ----------
texto = "No me gusta el servicio, pero la comida está muy buena."
print(analizar_sentimiento(texto))


{'sentimiento_global': -0.8, 'etiqueta': 'Negativo', 'detalles_por_span': [[-0.8], [-0.8]], 'polaridades_spans': [-0.8, -0.8], 'conjunciones': ['pero']}


In [4]:

# --- DICCIONARIO DE POLARIDAD ---
polaridad = {
    "bueno": 0.9,
    "excelente": 1.0,
    "positivo": 0.8,
    "agradable": 0.7,
    "bonito": 0.6,
    "malo": -0.8,
    "terrible": -1.0,
    "horrible": -1.0,
    "negativo": -0.7,
    "feo": -0.6,
    "caro": -0.4,
    "barato": 0.5,
    "rápido": 0.5,
    "lento": -0.5,
    "satisfecho": 0.8,
    "gusta": 0.9,
    "gustar": 0.9,
    "odio": -1.0,
    "odiar": -1.0,
    "encanta": 1.0,
    "encantar": 1.0,
    "mejor": 0.8,
    "peor": -0.8
}

# --- Normalización de adjetivos y formas ---
def normalizar_lemma(lemma):
    lemma = lemma.lower()
    if lemma.endswith("a") and lemma[:-1] + "o" in polaridad:
        return lemma[:-1] + "o"
    if lemma.endswith("as") and lemma[:-2] + "o" in polaridad:
        return lemma[:-2] + "o"
    if lemma.endswith("os") and lemma[:-2] + "o" in polaridad:
        return lemma[:-2] + "o"
    if lemma.endswith("es") and lemma[:-2] in polaridad:
        return lemma[:-2]
    return lemma

# --- Función de polaridad para un token individual ---
def sentimiento_token(token):
    lemma_norm = normalizar_lemma(token.lemma_)
    base = polaridad.get(lemma_norm)
    if base is None:
        return 0

    score = base

    # --- NEGACIÓN ---
    for child in token.children:
        if child.dep_ == "neg":
            score *= -1

    # --- INTENSIFICADORES ---
    for child in token.children:
        if child.dep_ == "advmod":
            if child.lemma_ in {"muy", "bastante", "demasiado"}:
                score *= 1.5
            elif child.lemma_ in {"poco"}:
                score *= 0.5

    return score

# --- División por conjunciones adversativas ---
def dividir_por_conjunciones(texto):
    doc = nlp(texto)
    suboraciones = []
    start = 0
    conjunciones = []
    for token in doc:
        if token.text.lower() in {"pero", "aunque", "sin embargo"}:
            sub = doc[start:token.i].text.strip(", ")
            if sub:
                suboraciones.append(sub)
            conjunciones.append(token.text.lower())
            start = token.i + 1
    final = doc[start:].text.strip(", ")
    if final:
        suboraciones.append(final)
    return suboraciones, conjunciones

# --- Análisis de sentimiento completo ---
def analizar_sentimiento(texto):
    suboraciones, conjunciones = dividir_por_conjunciones(texto)
    if not suboraciones:
        suboraciones = [texto]

    polaridades_spans = []
    detalles_por_span = []

    for sub in suboraciones:
        doc = nlp(sub)
        scores = []
        for token in doc:
            p = sentimiento_token(token)
            if p != 0:
                scores.append(p)
        if scores:
            polaridades_spans.append(sum(scores) / len(scores))
        else:
            polaridades_spans.append(0)
        detalles_por_span.append(scores)

    # --- Composición según conjunciones ---
    if conjunciones:
        if any(c in {"pero", "sin embargo"} for c in conjunciones):
            sentimiento_global = polaridades_spans[-1]  # prioriza la última
        elif "aunque" in conjunciones:
            sentimiento_global = sum(polaridades_spans) / len(polaridades_spans)
        else:
            sentimiento_global = sum(polaridades_spans) / len(polaridades_spans)
    else:
        sentimiento_global = sum(polaridades_spans) / len(polaridades_spans)

    if sentimiento_global > 0.1:
        etiqueta = "Positivo"
    elif sentimiento_global < -0.1:
        etiqueta = "Negativo"
    else:
        etiqueta = "Neutro"

    return {
        "sentimiento_global": round(sentimiento_global, 2),
        "etiqueta": etiqueta,
        "detalles_por_span": detalles_por_span,
        "polaridades_spans": polaridades_spans,
        "conjunciones": conjunciones
    }


In [5]:
ej1 = "No me gusta el servicio, pero la comida está muy buena."
ej2 = "El producto es caro aunque funciona bien."
ej3 = "El diseño es bonito y la batería dura mucho."
ej4 = "No está nada mal el servicio."
ej5 = "la comida está muy buena."


for e in [ej1, ej2, ej3, ej4]:
    print(e)
    print(analizar_sentimiento(e))
    print()


No me gusta el servicio, pero la comida está muy buena.
{'sentimiento_global': 0, 'etiqueta': 'Neutro', 'detalles_por_span': [[0.9], []], 'polaridades_spans': [0.9, 0], 'conjunciones': ['pero']}

El producto es caro aunque funciona bien.
{'sentimiento_global': -0.2, 'etiqueta': 'Negativo', 'detalles_por_span': [[-0.4], []], 'polaridades_spans': [-0.4, 0], 'conjunciones': ['aunque']}

El diseño es bonito y la batería dura mucho.
{'sentimiento_global': 0.6, 'etiqueta': 'Positivo', 'detalles_por_span': [[0.6]], 'polaridades_spans': [0.6], 'conjunciones': []}

No está nada mal el servicio.
{'sentimiento_global': 0.0, 'etiqueta': 'Neutro', 'detalles_por_span': [[]], 'polaridades_spans': [0], 'conjunciones': []}

